## Analyses scripts for the manuscript "Conceptual but not Behavioral Polarization of Climate Change around the World"

### Install Libraries

In [25]:
# Core analysis 
import numpy as np
import os, sys
import pandas as pd
from scipy import stats
from scipy.special import comb
import statsmodels.api as sm
import statsmodels.formula.api as smf
from statsmodels.stats import anova
import bff
import statsmodels.api as sm
import statsmodels.formula.api as smf

import matplotlib.pyplot as plt
plt.rcdefaults()
import seaborn as sns 
sns.set(style="ticks", color_codes=True)
sns.set_style("white")
sns.set_style({'xtick.bottom': True, 'ytick.left': True})
colorref = ["gray", "royalblue", "crimson", "goldenrod", "mediumorchid", "seagreen"]

# iPython magic commands
%matplotlib notebook
%load_ext autoreload
%autoreload 2
%autosave 30

SMALL_SIZE = 12
MEDIUM_SIZE = 12
BIG_SIZE = 14

plt.rc('font', size=SMALL_SIZE)          # controls default text sizes
plt.rc('axes', titlesize=SMALL_SIZE)     # fontsize of the axes title
plt.rc('axes', labelsize=MEDIUM_SIZE)    # fontsize of the x and y labels
plt.rc('xtick', labelsize=SMALL_SIZE)    # fontsize of the tick labels
plt.rc('ytick', labelsize=SMALL_SIZE)    # fontsize of the tick labels
plt.rc('legend', fontsize=SMALL_SIZE)    # legend fontsize
plt.rc('figure', titlesize=BIG_SIZE)  # fontsize of the figure title
cust_palette = sns.color_palette("Paired")[6:10]
cust_palette = [cust_palette[i] for i in [1,0,3,2]]

def median_split(S):
    return S > S.median()

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


Autosaving every 30 seconds


### Import data and run data preprocessing

In [26]:
df = pd.read_excel("data60.xlsx")
df

,ResponseId,Country,cond,condName,Intervention_order,Intro_Timer,Intervention_Timer,BELIEFcc,POLICYcc,SHAREcc,...,Trust_sci1_1.1,control_timer2_First Click,control_timer2_Last Click,control_timer2_Page Submit,control_timer2_Click Count,Trust_sci2_1.1,control_timer3_First Click,control_timer3_Last Click,control_timer3_Page Submit,control_timer3_Click Count
0,R_0031kFvKFnFD4E9,Israel,7,PsychDistance,SocialmediasharingPipedText|PolicySupport|Beli...,4.801,104.031,92.75,86.888889,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,R_0059fW1EtKYsTWV,Brazil,5,CollectAction,BeliefinAnthrCC|SocialmediasharingPipedText|Po...,11.155,198.395,100.00,100.000000,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,R_00688n4RYG3zIel,Macedonia,10,DynamicNorm,SocialmediasharingPipedText|PolicySupport|Beli...,29.154,83.853,100.00,54.000000,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,R_0086kfWbVfurc1H,France,8,PluralIgnorance,PolicySupport|SocialmediasharingPipedText|Beli...,8.301,29.399,80.25,85.222222,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,R_00B8XyVmrMofqGB,UK,8,PluralIgnorance,SocialmediasharingPipedText|PolicySupport|Beli...,17.951,148.918,51.00,48.777778,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
54653,R_zZMtNlzas0E8RyN,Slovakia,9,LetterFuture,BeliefinAnthrCC|PolicySupport|Socialmediashari...,31.481,316.675,98.50,91.333333,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
54654,R_zZPllZfBJWjfSCZ,Switzerland,9,LetterFuture,SocialmediasharingPipedText|BeliefinAnthrCC|Po...,5.627,206.393,63.00,47.444444,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
54655,R_zZPOdDw0XWugcH7,Israel,10,DynamicNorm,PolicySupport|SocialmediasharingPipedText|Beli...,16.698,104.157,98.75,63.888889,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
54656,R_zZpwx0yCZNh32ql,Algeria,10,DynamicNorm,BeliefinAnthrCC|PolicySupport|Socialmediashari...,17.577,79.966,13.75,25.750000,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [28]:
# Global median split
# df["Ideology"] = (df["Politics2_1"] > df["Politics2_1"].median()).astype(int)

df['ide'] = (df['Politics2_1'] + df['Politics2_9'])/2

# Per-country median split
new = []
for _, sub in df.groupby("Country"):
    sub.loc[:, "Ideology"] = (sub.loc[:, "ide"] > sub.loc[:, "ide"].median()).astype(float)
    # Ensure NaN ideology when Politics2_1 is NaN
    sub.loc[sub.Politics2_1.isna(), "Ideology"] = np.NaN
    new.append(sub)
df = pd.concat(new)
# Replace 1 with "Conservative" and 0 with "Liberal"
df['Ideology'] = df['Ideology'].replace(1, 'Conservative').replace(0, 'Liberal')

In [29]:
#transform the datafram from wide format to long format

df_z = pd.melt(
    df.loc[:, ['ResponseId', "Country", "condName", "cond", "Ideology","ide", "BELIEFcc", "WEPTcc"]],
    id_vars=['ResponseId', "Country", "condName",  "Ideology","ide", "cond"],
    var_name="Type",
    value_name="score"
)
df_z_control=df_z.query('cond==1')
df_control = df.query('cond==1')

## Main manuscript analyses

In [31]:
import warnings
warnings.filterwarnings('ignore')

# Load R in Python
%load_ext rpy2.ipython

The rpy2.ipython extension is already loaded. To reload it, use:
  %reload_ext rpy2.ipython


In [32]:
%R library(lme4)
%R library(lmerTest)

array(['ordinal', 'lmerTest', 'lme4', 'Matrix', 'tools', 'stats',
       'graphics', 'grDevices', 'utils', 'datasets', 'methods', 'base'],
      dtype='<U9')

In [33]:
%R library(ordinal)

array(['ordinal', 'lmerTest', 'lme4', 'Matrix', 'tools', 'stats',
       'graphics', 'grDevices', 'utils', 'datasets', 'methods', 'base'],
      dtype='<U9')

# 1.1 Climate Belief

In [36]:
dfc_belief = pd.melt(
    df_control.loc[:, ['ResponseId', 'Country', 'condName', 'ide','Ideology','Income','Edu','Age','Gender', "WEPTcc",'Belief1', 'Belief2', 'Belief3', 'Belief4']],
    id_vars=['ResponseId', 'Country', 'condName', 'ide','Ideology','Income','Edu','Age','Gender', 'WEPTcc'],
    var_name="Item",
    value_name="Belief"
)
dfc_belief['condName'] = dfc_belief['condName'].replace(['Control'], 'aControl')
dfc_belief = dfc_belief.dropna()
%Rpush dfc_belief

In [37]:
%%R

M1 <- lmer(Belief ~ ide + (1 | Country) + (1 | ResponseId) + (1 | Item), data = dfc_belief)
print(summary(M1))

Linear mixed model fit by REML. t-tests use Satterthwaite's method [
lmerModLmerTest]
Formula: Belief ~ ide + (1 | Country) + (1 | ResponseId) + (1 | Item)
   Data: dfc_belief

REML criterion at convergence: 132060.9

Scaled residuals: 
    Min      1Q  Median      3Q     Max 
-6.2864 -0.2870  0.0880  0.3195  5.9382 

Random effects:
 Groups     Name        Variance Std.Dev.
 ResponseId (Intercept) 507.7780 22.5339 
 Country    (Intercept)  33.7523  5.8097 
 Item       (Intercept)   0.3565  0.5971 
 Residual               148.0690 12.1684 
Number of obs: 15511, groups:  ResponseId, 3881; Country, 57; Item, 4

Fixed effects:
              Estimate Std. Error         df t value Pr(>|t|)    
(Intercept)   90.96277    1.27233  161.66940   71.49   <2e-16 ***
ide           -0.21449    0.01635 3870.76438  -13.12   <2e-16 ***
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1

Correlation of Fixed Effects:
    (Intr)
ide -0.663


In [14]:
%%R

M1 <- lmer(Belief ~ ide + Income + Edu + Age + as.factor(Gender) + (1 | Country) + (1 | ResponseId) + (1 | Item), data = dfc_belief)
print(summary(M1))

Linear mixed model fit by REML. t-tests use Satterthwaite's method [
lmerModLmerTest]
Formula: Belief ~ ide + Income + Edu + Age + as.factor(Gender) + (1 |  
    Country) + (1 | ResponseId) + (1 | Item)
   Data: dfc_belief

REML criterion at convergence: 131999.9

Scaled residuals: 
    Min      1Q  Median      3Q     Max 
-6.2904 -0.2866  0.0876  0.3217  5.9375 

Random effects:
 Groups     Name        Variance Std.Dev.
 ResponseId (Intercept) 501.2296 22.3882 
 Country    (Intercept)  30.7853  5.5485 
 Item       (Intercept)   0.3567  0.5973 
 Residual               148.0697 12.1684 
Number of obs: 15511, groups:  ResponseId, 3881; Country, 57; Item, 4

Fixed effects:
                     Estimate Std. Error         df t value Pr(>|t|)    
(Intercept)          86.91763    2.33947 1376.19935  37.153  < 2e-16 ***
ide                  -0.20313    0.01643 3864.36206 -12.360  < 2e-16 ***
Income                0.13926    0.17875 3493.83954   0.779 0.435981    
Edu                   1.34019

# 1.2 Climate Policy Support

In [38]:
dfc_policy = pd.melt(
    df_control.loc[:, ['ResponseId', 'Country', 'condName', 'Income', 'Edu', 'Age', 'Gender', 'ide','Ideology', "WEPTcc",'Policy1', 'Policy2', 'Policy3', 'Policy4','Policy5', 'Policy6', 'Policy7', 'Policy8', 'Policy9']],
    id_vars=['ResponseId', 'Country', 'condName', 'Income', 'Edu', 'Age', 'Gender', 'ide','Ideology', 'WEPTcc'],
    var_name="Item",
    value_name="Policy"
)
dfc_policy = dfc_policy.dropna()
%Rpush dfc_policy

In [18]:
%%R

M1 <- lmer(Policy ~ ide + (1 | Country) + (1 | ResponseId) + (1 | Item), data = dfc_policy)
print(summary(M1))

Linear mixed model fit by REML. t-tests use Satterthwaite's method [
lmerModLmerTest]
Formula: Policy ~ ide + (1 | Country) + (1 | ResponseId) + (1 | Item)
   Data: dfc_policy

REML criterion at convergence: 309392.1

Scaled residuals: 
    Min      1Q  Median      3Q     Max 
-4.5615 -0.5313  0.0611  0.6249  4.2178 

Random effects:
 Groups     Name        Variance Std.Dev.
 ResponseId (Intercept) 310.37   17.617  
 Country    (Intercept)  20.46    4.523  
 Item       (Intercept) 158.14   12.575  
 Residual               412.49   20.310  
Number of obs: 34019, groups:  ResponseId, 3870; Country, 57; Item, 9

Fixed effects:
              Estimate Std. Error         df t value Pr(>|t|)    
(Intercept)   76.97180    4.30635    8.88911  17.874 2.84e-08 ***
ide           -0.13037    0.01325 3848.49638  -9.837  < 2e-16 ***
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1

Correlation of Fixed Effects:
    (Intr)
ide -0.159


In [23]:
%%R

M1 <- lmer(Policy ~ ide + Income + Edu + Age + as.factor(Gender) + (1 | Country) + (1 | ResponseId) + (1 | Item), data = dfc_policy)
print(summary(M1))

Linear mixed model fit by REML. t-tests use Satterthwaite's method [
lmerModLmerTest]
Formula: Policy ~ ide + Income + Edu + Age + as.factor(Gender) + (1 |  
    Country) + (1 | ResponseId) + (1 | Item)
   Data: dfc_policy

REML criterion at convergence: 309357.5

Scaled residuals: 
    Min      1Q  Median      3Q     Max 
-4.5628 -0.5302  0.0616  0.6257  4.2242 

Random effects:
 Groups     Name        Variance Std.Dev.
 ResponseId (Intercept) 308.12   17.553  
 Country    (Intercept)  19.28    4.391  
 Item       (Intercept) 158.06   12.572  
 Residual               412.50   20.310  
Number of obs: 34019, groups:  ResponseId, 3870; Country, 57; Item, 9

Fixed effects:
                     Estimate Std. Error         df t value Pr(>|t|)    
(Intercept)          70.88835    4.59523   11.54657  15.427 4.62e-09 ***
ide                  -0.12390    0.01336 3841.50788  -9.272  < 2e-16 ***
Income               -0.13793    0.14592 3462.31125  -0.945   0.3446    
Edu                   1.97344

# 1.3 Behavioral Climate Polarization

In [39]:
df_WEPT = df_control[["WEPTcc", "ide", "Income", "Edu", "Age", "Gender", "Country"]]
%Rpush df_WEPT

In [26]:
%%R

M <- clmm(as.factor(WEPTcc) ~ ide + (1|Country), data = df_WEPT, threshold = "equidistant",  doFit = TRUE)
summary(M)

Cumulative Link Mixed Model fitted with the Laplace approximation

formula: as.factor(WEPTcc) ~ ide + (1 | Country)
data:    df_WEPT

 link  threshold   nobs logLik   AIC      niter    max.grad cond.H 
 logit equidistant 3897 -6131.70 12271.39 173(526) 8.25e-03 2.1e+04

Random effects:
 Groups  Name        Variance Std.Dev.
 Country (Intercept) 0.1715   0.4142  
Number of groups:  Country 57 

Coefficients:
     Estimate Std. Error z value Pr(>|z|)
ide -0.001002   0.001345  -0.745    0.456

Threshold coefficients:
             Estimate Std. Error z value
threshold.1 -1.451749   0.099108  -14.65
spacing      0.184235   0.004932   37.36
(726 observations deleted due to missingness)


In [27]:
%%R

M <- clmm(as.factor(WEPTcc) ~ ide + Income + Edu + Age + as.factor(Gender) + (1|Country), data = df_WEPT, threshold = "equidistant",  doFit = TRUE)
summary(M)

Cumulative Link Mixed Model fitted with the Laplace approximation

formula: as.factor(WEPTcc) ~ ide + Income + Edu + Age + as.factor(Gender) +  
    (1 | Country)
data:    df_WEPT

 link  threshold   nobs logLik   AIC      niter     max.grad cond.H 
 logit equidistant 3887 -6020.69 12061.38 941(2835) 1.65e-01 2.7e+05

Random effects:
 Groups  Name        Variance Std.Dev.
 Country (Intercept) 0.17     0.4123  
Number of groups:  Country 57 

Coefficients:
                    Estimate Std. Error z value Pr(>|z|)    
ide                -0.001560   0.001380  -1.131   0.2582    
Income              0.017750   0.014713   1.206   0.2277    
Edu                 0.028320   0.044727   0.633   0.5266    
Age                 0.028429   0.002248  12.644  < 2e-16 ***
as.factor(Gender)2  0.347524   0.064288   5.406 6.45e-08 ***
as.factor(Gender)3 -0.481125   0.442616  -1.087   0.2770    
as.factor(Gender)4  0.636824   0.373372   1.706   0.0881 .  
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05

In [45]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
df[['BELIEFcc', 'WEPTcc']] = scaler.fit_transform(df[['BELIEFcc', 'WEPTcc']])

df_z = pd.melt(
    df.loc[:, ['ResponseId', "Country", "condName", "cond", "Ideology","ide", "BELIEFcc", "WEPTcc"]],
    id_vars=['ResponseId', "Country", "condName",  "Ideology","ide", "cond"],
    var_name="Type",
    value_name="score"
)

df_z_control=df_z.query('cond==1')

In [46]:
%Rpush df_z_control

In [47]:
%%R

M1 <- lmer(score ~ Type*ide + (1 | Country) + (1 | ResponseId), data = df_z_control)
print(summary(M1))

Linear mixed model fit by REML. t-tests use Satterthwaite's method [
lmerModLmerTest]
Formula: score ~ Type * ide + (1 | Country) + (1 | ResponseId)
   Data: df_z_control

REML criterion at convergence: 21809.9

Scaled residuals: 
    Min      1Q  Median      3Q     Max 
-3.4841 -0.7766  0.3684  0.7621  1.6904 

Random effects:
 Groups     Name        Variance Std.Dev.
 ResponseId (Intercept) 0.08776  0.2963  
 Country    (Intercept) 0.02852  0.1689  
 Residual               0.86601  0.9306  
Number of obs: 7788, groups:  ResponseId, 3897; Country, 57

Fixed effects:
                 Estimate Std. Error         df t value Pr(>|t|)    
(Intercept)     4.653e-01  4.506e-02  4.165e+02  10.325  < 2e-16 ***
TypeWEPTcc     -3.093e-01  5.073e-02  3.891e+03  -6.097 1.19e-09 ***
ide            -8.845e-03  6.720e-04  7.648e+03 -13.163  < 2e-16 ***
TypeWEPTcc:ide  8.332e-03  8.885e-04  3.891e+03   9.378  < 2e-16 ***
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1

Correlation o

# 1.4 Mechanism

In [40]:
%%R

M <- clmm(as.factor(WEPTcc) ~ ide*Belief + (1|Country), data = dfc_belief, threshold = "equidistant")
summary(M)

Cumulative Link Mixed Model fitted with the Laplace approximation

formula: as.factor(WEPTcc) ~ ide * Belief + (1 | Country)
data:    dfc_belief

 link  threshold   nobs  logLik    AIC      niter    max.grad cond.H 
 logit equidistant 15511 -24176.72 48365.44 137(561) 1.29e+04 7.7e+08

Random effects:
 Groups  Name        Variance Std.Dev.
 Country (Intercept) 1        1       
Number of groups:  Country 57 

Coefficients:
             Estimate Std. Error z value Pr(>|z|)    
ide         0.0096209  0.0018815   5.113 3.16e-07 ***
Belief      0.0144224  0.0013937  10.348  < 2e-16 ***
ide:Belief -0.0001095  0.0000221  -4.954 7.26e-07 ***
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1

Threshold coefficients:
             Estimate Std. Error z value
threshold.1 -0.168326   0.180743  -0.931
spacing      0.179952   0.002425  74.209


In [41]:
%%R

M <- clmm(as.factor(WEPTcc) ~ ide*Belief + Income + Edu + Age + as.factor(Gender) + (1|Country), data = dfc_belief, threshold = "equidistant")
summary(M)

Cumulative Link Mixed Model fitted with the Laplace approximation

formula: 
as.factor(WEPTcc) ~ ide * Belief + Income + Edu + Age + as.factor(Gender) +  
    (1 | Country)
data:    dfc_belief

 link  threshold   nobs  logLik    AIC      niter     max.grad cond.H 
 logit equidistant 15511 -23802.46 47628.92 375(1518) 1.24e+04 1.2e+09

Random effects:
 Groups  Name        Variance Std.Dev.
 Country (Intercept) 1        1       
Number of groups:  Country 57 

Coefficients:
                     Estimate Std. Error z value Pr(>|z|)    
ide                 5.829e-03  1.903e-03   3.063 0.002189 ** 
Belief              1.367e-02  1.410e-03   9.696  < 2e-16 ***
Income              8.865e-03  7.586e-03   1.168 0.242614    
Edu                 9.417e-03  2.261e-02   0.416 0.677063    
Age                 2.811e-02  1.159e-03  24.267  < 2e-16 ***
as.factor(Gender)2  2.710e-01  3.260e-02   8.313  < 2e-16 ***
as.factor(Gender)3 -3.423e-01  2.354e-01  -1.454 0.145899    
as.factor(Gender)4  5.267e-

# 2. How do climate interventions impact the conceptual and behavioral outcomes of people along the ideological spectrum?

## 2.1 Interventions’ Impact on Beliefs

In [48]:
df_belief = pd.melt(
    df.loc[:, ['ResponseId', 'Country', 'condName', 'ide','Ideology', "WEPTcc", 'POLICYcc','Belief1', 'Belief2', 'Belief3', 'Belief4']],
    id_vars=['ResponseId', 'Country', 'condName', 'ide','Ideology', 'WEPTcc'],
    var_name="Item",
    value_name="Belief"
)
df_belief['condName'] = df_belief['condName'].replace(['Control'], 'aControl')
df_belief = df_belief.dropna()
%Rpush df_belief

## TABLE 2: Belief by Ideology by Condition

In [43]:
%%R

M1 <- lmer(Belief ~ condName*Ideology + (1 | Country) + (1 | ResponseId), data = df_belief)
print(summary(M1))

Linear mixed model fit by REML. t-tests use Satterthwaite's method [
lmerModLmerTest]
Formula: Belief ~ condName * Ideology + (1 | Country) + (1 | ResponseId)
   Data: df_belief

REML criterion at convergence: 1590708

Scaled residuals: 
    Min      1Q  Median      3Q     Max 
-6.4166 -0.2753  0.0909  0.2990  6.0797 

Random effects:
 Groups     Name        Variance Std.Dev.
 ResponseId (Intercept) 505.46   22.483  
 Country    (Intercept)  37.66    6.137  
 Residual               141.51   11.896  
Number of obs: 187733, groups:  ResponseId, 46978; Country, 57

Fixed effects:
                                           Estimate Std. Error         df
(Intercept)                               7.626e+01  9.778e-01  1.089e+02
condNameBindingMoral                      7.912e-01  7.430e-01  4.688e+04
condNameCollectAction                     2.451e+00  7.444e-01  4.689e+04
condNameDynamicNorm                       4.104e-01  7.447e-01  4.689e+04
condNameFutureSelfCont                    1.57

R[write to console]: 
Correlation matrix not shown by default, as p = 24 > 12.
Use print(summary(M1), correlation=TRUE)  or
    vcov(summary(M1))        if you need it




## TABLE 3: Policy by Ideology by Condition

In [50]:
df_policy = pd.melt(
    df.loc[:, ['ResponseId', 'Country', 'condName', 'Income', 'Edu', 'Age', 'Gender', 'ide','Ideology', "WEPTcc",'Policy1', 'Policy2', 'Policy3', 'Policy4','Policy5', 'Policy6', 'Policy7', 'Policy8', 'Policy9']],
    id_vars=['ResponseId', 'Country', 'condName', 'Income', 'Edu', 'Age', 'Gender', 'ide','Ideology', 'WEPTcc'],
    var_name="Item",
    value_name="Policy"
)
df_policy['condName'] = df_policy['condName'].replace(['Control'], 'aControl')
%Rpush df_policy

In [51]:
%%R

M1 <- lmer(Policy ~ condName*Ideology + (1 | Country) + (1 | ResponseId), data = df_policy)
print(summary(M1))

Linear mixed model fit by REML. t-tests use Satterthwaite's method [
lmerModLmerTest]
Formula: Policy ~ condName * Ideology + (1 | Country) + (1 | ResponseId)
   Data: df_policy

REML criterion at convergence: 3912486

Scaled residuals: 
    Min      1Q  Median      3Q     Max 
-3.6995 -0.5080  0.1837  0.6157  3.3734 

Random effects:
 Groups     Name        Variance Std.Dev.
 ResponseId (Intercept) 294.89   17.172  
 Country    (Intercept)  25.44    5.044  
 Residual               563.40   23.736  
Number of obs: 417640, groups:  ResponseId, 47625; Country, 57

Fixed effects:
                                           Estimate Std. Error         df
(Intercept)                                 68.5106     0.8019   109.0932
condNameBindingMoral                         0.3490     0.6070 47369.9167
condNameCollectAction                        2.5799     0.6076 47349.9647
condNameDynamicNorm                          0.6126     0.6082 47403.0738
condNameFutureSelfCont                       1

R[write to console]: 
Correlation matrix not shown by default, as p = 24 > 12.
Use print(summary(M1), correlation=TRUE)  or
    vcov(summary(M1))        if you need it




## TABLE 4: Action by Ideology by Condition

In [52]:
df_WEPTall = df[["WEPTcc", "Ideology", "Income", "Edu", "Age", "Gender", "Country", "condName"]]
df_WEPTall['condName'] = df_WEPTall['condName'].replace(['Control'], 'aControl')
%Rpush df_WEPTall

In [53]:
%%R

M <- clmm(as.factor(WEPTcc) ~ condName*Ideology + (1|Country), data = df_WEPTall, threshold = "equidistant")
summary(M)

Cumulative Link Mixed Model fitted with the Laplace approximation

formula: as.factor(WEPTcc) ~ condName * Ideology + (1 | Country)
data:    df_WEPTall

 link  threshold   nobs  logLik    AIC       niter       max.grad cond.H 
 logit equidistant 47851 -77487.54 155027.09 6532(19897) 1.83e-01 3.2e+04

Random effects:
 Groups  Name        Variance Std.Dev.
 Country (Intercept) 0.1744   0.4176  
Number of groups:  Country 57 

Coefficients:
                                         Estimate Std. Error z value Pr(>|z|)
condNameBindingMoral                     -0.04434    0.06108  -0.726 0.467908
condNameCollectAction                    -0.15373    0.06079  -2.529 0.011436
condNameDynamicNorm                      -0.07512    0.06088  -1.234 0.217231
condNameFutureSelfCont                   -0.21095    0.06390  -3.301 0.000962
condNameLetterFuture                     -0.31102    0.06443  -4.827 1.38e-06
condNameNegativeEmotions                 -0.35150    0.06146  -5.719 1.07e-08
condNamePlur